In [1]:
from pathlib import Path
import os, shutil
import pandas as pd
from typing import Tuple, List, Dict
import argparse

from steps import (
    SplitGuaranteeingAllClassesPerSplit,
    BalanceToMinimumClass,
    BalanceToMinimumClassAndUser,
    FilterByCommonRows,
)

from pipelines import match_columns, pipelines

from readers import read_hiacc_smartphone


# Set the seed for reproducibility
import numpy as np
import random
import traceback

np.random.seed(42)
random.seed(42)
# pd.np.random.seed(42)


In [2]:
dataset_readers: Dict[str, callable] = {
    "HIAAC": read_hiacc_smartphone,
}
dataset = "HIAAC" 
reader = dataset_readers[dataset]
path = Path("/home/maria/git/DAGHAR/data/original/HIAAC")
raw_dataset = reader(path)

Iniciando leitura do dataset HIACC...
Processamento concluído.


In [3]:
print(raw_dataset.shape)
raw_dataset.head()

(198000, 12)


,timestamp-server-accel,timestamp-server-gyro,accel-x,accel-y,accel-z,gyro-x,gyro-y,gyro-z,position,activity code,trial,user
0,0,0,0.531513,-9.809343,-0.404620,0.011988,0.003207,0.010156,Bolso_direito,0,0,1
1,9,9,0.531513,-9.838074,-0.414197,0.010767,0.000764,0.012599,Bolso_direito,0,0,1
2,17,21,0.529119,-9.838074,-0.423774,0.013210,-0.005345,0.015653,Bolso_direito,0,0,1
3,28,31,0.577003,-9.828497,-0.359130,0.015043,-0.008399,0.015043,Bolso_direito,0,0,1
4,39,42,0.588974,-9.826103,-0.318429,0.024816,-0.009010,0.016264,Bolso_direito,0,0,1


In [4]:
raw_dataset.columns.tolist()


['timestamp-server-accel',
 'timestamp-server-gyro',
 'accel-x',
 'accel-y',
 'accel-z',
 'gyro-x',
 'gyro-y',
 'gyro-z',
 'position',
 'activity code',
 'trial',
 'user']

# Aplicando Pipeline 

In [5]:

# %% [code]
# Verifica quais pipelines estão disponíveis
print("Pipelines disponíveis:")
available_keys = list(pipelines.keys())
print(available_keys)

Pipelines disponíveis:
['KuHar', 'MotionSense', 'WISDM', 'UCI', 'RealWorld', 'HIAAC']


In [6]:
# Aplicando os pipelines
# Supondo que no pipeline "HIAAC" exista a função "raw_dataset" e "standardized"
df_raw = pipelines["HIAAC"]["raw_dataset"](raw_dataset)
df_standardized = pipelines["HIAAC"]["standardized"](raw_dataset)

print("Formato do dataset processado (raw):", df_raw.shape)
display(df_raw.head())

print("Formato do dataset processado (standardized):", df_standardized.shape)
display(df_standardized.head())

Executing Windowize


Creating windows: 100%|██████████| 660/660 [01:31<00:00,  7.20it/s]


Executing AddStandardActivityCode
Executing ButterworthFilter
Executing ResamplerPoly


Resampling: 100%|██████████| 660/660 [00:04<00:00, 163.24it/s]


Executing Windowize


Creating windows: 100%|██████████| 660/660 [00:22<00:00, 29.87it/s]


Executing AddStandardActivityCode
Formato do dataset processado (raw): (660, 1808)


,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,gyro-z-298,gyro-z-299,position,activity code,timestamp-server-accel,trial,timestamp-server-gyro,user,window,standard activity code
0,0.531513,0.531513,0.529119,0.577003,0.588974,0.557849,0.52433,0.483629,0.47884,0.591368,...,0.068799,0.092622,Bolso_direito,0,0,0,0,1,0,NaN
1,-9.543588,-9.581895,-9.562741,-9.562741,-9.56753,-9.519646,-9.50528,-9.510069,-9.565135,-9.593865,...,0.103771,0.101327,Mochila,0,0,0,0,1,0,NaN
2,0.002394,0.349553,0.507571,0.584185,0.531513,0.349553,0.181959,-0.189142,-0.040701,0.167594,...,0.00649,0.002825,Bolso_direito,0,3020,1,3037,1,0,NaN
3,-9.498097,-9.615414,-9.701605,-9.792584,-9.924265,-9.988909,-10.027216,-10.012851,-9.986514,-9.924265,...,0.012752,0.015195,Mochila,0,3041,1,3038,1,0,NaN
4,0.543484,0.562637,0.550666,0.488417,0.471658,0.519542,0.538695,0.545878,0.500388,0.521936,...,-0.005116,-0.00817,Bolso_direito,0,6044,2,6072,1,0,NaN


Formato do dataset processado (standardized): (660, 369)


,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,gyro-z-59,position,index,timestamp-server-accel,activity code,trial,timestamp-server-gyro,user,window,standard activity code
0,0.043939,0.058694,0.099691,0.10559,0.108888,0.096158,0.05973,0.056849,-0.09334,-0.068499,...,-0.034904,Bolso_direito,0,0,0,0,0,1,0,NaN
1,-2.867676,-4.520543,-3.774927,-3.713099,-3.313451,-3.15736,-2.779865,-2.613535,-2.305281,-2.108418,...,0.102453,Mochila,99000,0,0,0,0,1,0,NaN
2,-0.180496,-0.366092,-0.566072,-0.530784,-0.536695,-0.325834,0.496649,-0.347638,1.147642,1.126622,...,0.007245,Bolso_direito,300,3020,0,1,3037,1,0,NaN
3,-0.08915,-0.457753,-0.49193,-0.305291,-0.199441,-0.092278,-0.361301,-0.318514,0.025984,-0.401857,...,0.009784,Mochila,99300,3041,0,1,3038,1,0,NaN
4,0.009302,-0.015479,0.026727,0.015229,0.000781,0.004393,0.012528,0.075865,-0.005746,0.070646,...,-0.002204,Bolso_direito,600,6044,0,2,6072,1,0,NaN


In [7]:
pd.set_option('display.max_columns', None)
df_raw

accel-x-0  accel-x-1  accel-x-2  accel-x-3 accel-x-4 accel-x-5  \
0     0.531513   0.531513   0.529119   0.577003  0.588974  0.557849   
1    -9.543588  -9.581895  -9.562741  -9.562741  -9.56753 -9.519646   
2     0.002394   0.349553   0.507571   0.584185  0.531513  0.349553   
3    -9.498097  -9.615414  -9.701605  -9.792584 -9.924265 -9.988909   
4     0.543484   0.562637   0.550666   0.488417  0.471658  0.519542   
..         ...        ...        ...        ...       ...       ...   
655 -10.321703 -10.733505 -10.692804 -10.290578 -9.749489 -9.189245   
656   1.467646   2.013524   2.420538    2.65517  2.585738  2.312799   
657 -13.029545 -12.996026 -12.328044  -10.74787 -9.730335 -8.751106   
658   3.849876   3.531447   3.397372   2.815581  1.240196 -0.486023   
659  -8.990526  -8.660127  -8.617031  -8.942642 -9.067142 -9.256283   

     accel-x-6  accel-x-7  accel-x-8  accel-x-9 accel-x-10 accel-x-11  \
0      0.52433   0.483629    0.47884   0.591368   0.620098    0.59855   
1     -9.50528  -9.510069  -9.565135  -9.593865  -9.603442    -9.5795   
2     0.181959  -0.189142  -0.040701   0.167594   0.390255   0.512359   
3   -10.027216 -10.012851  -9.986514  -9.924265  -9.912294  -9.871593   
4     0.538695   0.545878   0.500388   0.521936   0.562637   0.603339   
..         ...        ...        ...        ...        ...        ...   
655  -8.698434  -8.233959  -7.939472  -7.860464  -7.767089  -7.798214   
656   2.183512   1.956063   1.910573   1.996764   2.075773    2.15957   
657  -8.746318  -9.531616 -10.441413 -11.367969 -12.261006 -12.605771   
658  -1.673547  -1.807622  -1.647211  -0.703895   0.205901    1.00317   
659  -9.105449  -9.074324  -9.912294 -10.795754 -11.176433 -10.185233   

    accel-x-12 accel-x-13 accel-x-14 accel-x-15 accel-x-16 accel-x-17  \
0     0.622492   0.639252   0.605733   0.553061   0.500388   0.509965   
1     -9.56753  -9.529222  -9.534011  -9.512463  -9.507674   -9.50528   
2      0.35913  -0.153229  -0.316035  -0.201113   0.129287   0.414197   
3    -9.900323  -9.905111  -9.914688  -9.842862  -9.692028   -9.62499   
4     0.586579   0.548272   0.526724   0.500388   0.521936   0.567426   
..         ...        ...        ...        ...        ...        ...   
655  -7.944261  -8.794202  -9.124602  -9.172486  -9.222764  -9.131784   
656   2.367866   2.339135   2.059013   1.934515   2.161964   3.031059   
657   -12.9242 -12.835614 -13.467684  -13.49402  -12.38311 -10.891522   
658    1.63524   1.817199   1.745373   1.419762   1.118092   0.792481   
659  -8.801385  -7.987356  -6.713641   -6.32578  -5.983409  -5.787085   

    accel-x-18 accel-x-19 accel-x-20 accel-x-21 accel-x-22 accel-x-23  \
0     0.577003   0.656011   0.677559   0.612916    0.56982   0.543484   
1    -9.522039  -9.512463  -9.574712  -9.586683  -9.584289  -9.584289   
2     0.526724   0.390255   0.100556   0.035913   0.007183   0.102951   
3     -9.62499  -9.648932   -9.69921  -9.639356  -9.663298  -9.605837   
4     0.543484   0.538695   0.538695   0.533907   0.536301   0.517148   
..         ...        ...        ...        ...        ...        ...   
655  -8.990526  -8.734347  -8.549993  -8.475773  -8.851663  -9.280225   
656   3.296816    3.25372   2.451662    0.40462  -0.890643  -1.762132   
657  -7.903559   -7.25473  -6.589142  -6.368876  -6.469432  -6.596325   
658   0.500388   0.229843  -0.014365   -0.23942  -0.617704  -0.744597   
659  -6.021717  -6.560412  -7.127838  -7.398383  -7.728782  -7.865252   

    accel-x-24 accel-x-25 accel-x-26 accel-x-27 accel-x-28 accel-x-29  \
0     0.548272    0.54109   0.555455   0.586579   0.581791   0.579397   
1    -9.581895  -9.560347  -9.545981  -9.498097  -9.493309  -9.522039   
2     0.203507   0.280122   0.385466   0.605733   1.077391    1.44131   
3    -9.598654  -9.510069   -9.50528  -9.548376  -9.658509  -9.732729   
4     0.509965   0.526724    0.52433   0.560243   0.565032   0.531513   
..         ...        ...        ...        ...        ...        ...   
655  -9.78